# Bim File

variant identifier and minor allele

In [2]:
import pandas as pd
df_bim=pd.read_csv('Data/DataS1.bim', sep='\t', header=None)
# df_bim = 1 and 4 columns
df_bim = df_bim[[1, 4]]

df_bim.rename(columns={1: 'VariantID', 4: 'minor'}, inplace=True)

In [3]:
# conver df_bim['variantID'] to a list
columns= df_bim['VariantID'].to_list()
columns.insert(0, 'familyID')
columns.insert(1, 'individualID')
columns.insert(2, 'fatherID')
columns.insert(3, 'motherID')
columns.insert(4,'Sex')
columns.insert(5, 'date')

In [4]:
minor=df_bim['minor'].to_list()
minor.insert(0, 'None')
minor.insert(1, 'None')
minor.insert(2, 'None')
minor.insert(3, 'None')
minor.insert(4,'None')
minor.insert(5, 'None')

# Ped File

familly id and individual id and allels

## Split the ped file by familly id

In [5]:
! rm -r Data/out/*

rm: cannot remove 'Data/out/*': No such file or directory


In [6]:
import os
 
def split_and_convert(file_path):
    # Create a directory to store output files
    os.makedirs('Data/out', exist_ok=True)
    populations = []
    with open(file_path, 'r') as ped_file:
        for row in ped_file:
            row_data = row.strip().split()
            populations.append(row_data[0])
            pedigree = row_data[0]
            with open(f'Data/out/{pedigree}.txt', 'a') as out_file:
                # Join the pairs of alleles in each row
                new_fields = row_data[:6]
                for i in range(6, len(row_data), 2):
                    new_fields.append(row_data[i] + row_data[i+1])
                # Write the new fields to the output file
                out_file.write(' '.join(new_fields) + '\n')
        
    return list(set(populations))

file_path = 'Data/DataS1.ped'

populations = split_and_convert(file_path)
print(populations)



In [ ]:
dfeuro = pd.read_excel(
    'Data/Eurasian - Dataset_tims.xlsx', sheet_name='Eurasian')
dfeuro = pd.DataFrame(dfeuro)
dfeuro.rename(columns={
              'Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]': 'date'}, inplace=True)

# date_range is a list of the range of dates from 0 to 12000 years ago in 1000 year intervals
date_range = list(range(0, 12000, 1000)) 
# if dfeuro['date'] is in the range of date_range, then dfeuro['cat_date'] is the index of the range
dfeuro['cat_date']=pd.cut(dfeuro['date'], date_range, labels=range(0, len(date_range)-1))


In [ ]:

def freq(pop_path, country):
    '''This function takes a population file and returns a dictionary of the frequency of each variant'''
    df = pd.read_csv(pop_path, sep=' ', header=None)
    df.columns = columns
    df.date = 0
    # print(len(df))
    # locate df['individualID'] in dfeuro['Master ID'] and add df['cat_date'] to the df[date]
    for i in range(0, len(df)):
        for j in range(0, len(dfeuro)):
            if df['individualID'][i] == dfeuro['Master ID'][j]:
                df['date'][i] = dfeuro['cat_date'][j]
  
    # get unique dates in df['date'] and store in a list and remove None
    date = df['date'].unique()
    date = date.tolist()
    if 0 in date:
        date.remove(0)
    

    countryfreq = {}
    
    for d in date:
        freqdate= []
        df2 = df[df['date'] == d]
        for i in range(6, len(df.columns)):
            minorAllele = minor[i]
            count = 0
            total = len(df2)*2
            for j in range(0, len(df2)):
                # count occurrences of the minor allele in column i
                count += str(df2.iloc[j, i]).count(str(minorAllele))
            freqdate.append(round(count/total),3)
            
        name= f'{country}_{d}'
        countryfreq[name] = freqdate
        
    return countryfreq


# ! rm Data/DataS1_freq.txt
# counter = 0
# with open('Data/DataS1_freq.txt', 'a') as f:
#     for country in populations:
#         print(country)
#         pop_path=f'Data/out/{country}.txt'
#         freq_results = freq(pop_path, country)
#         for key, value in freq_results.items():
#             str_values = [str(val) for val in value]  # convert float values to strings
#             f.write('%s\t%s\n' % (key, '\t'.join(str_values)))
#         counter += 1
#         print(counter, 'of', len(populations), 'done')


In [1]:
import gzip

# Open the input and output files
with open('Data/DataS1_freq.txt', 'rb') as f_in, gzip.open('Data/DataS1_freq.txt.gz', 'wb') as f_out:
    # Copy the contents of the input file to the output file using gzip compression
    f_out.writelines(f_in)
